## Import

In [1]:
# !pip install transformers datasets

In [2]:
import pandas as pd
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch import functional as F
# from torchsummary import summary

# 모델, Tokenizer Load
from transformers import AutoModel, AutoTokenizer

# 데이터셋 Load from Summarize_from_feedvback, Huggingface 
from datasets import load_dataset

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


## Tokenizer

In [4]:
tokenizer = AutoTokenizer.from_pretrained("allenai/led-large-16384-arxiv")

## DataLoad

In [6]:
# Feedback 포함 데이터
# data_feedback = load_dataset("openai/summarize_from_feedback", 'comparisons')

# Text Summary 데이터 
## CNN, TL;DR, Daily Mail
data_textsum = load_dataset("openai/summarize_from_feedback", 'axis')

Found cached dataset summarize_from_feedback (/root/.cache/huggingface/datasets/openai___summarize_from_feedback/axis/0.0.0/483f970ceb55b926b0a087ef4f678ab1b089bc8174a107a452c6152e88af7ff0)


  0%|          | 0/2 [00:00<?, ?it/s]

### Data Preprocessing

- Original Text와 Summarized Text 분류


In [9]:
data_textsum

DatasetDict({
    test: Dataset({
        features: ['info', 'summary', 'worker', 'batch', 'split'],
        num_rows: 6312
    })
    validation: Dataset({
        features: ['info', 'summary', 'worker', 'batch', 'split'],
        num_rows: 8585
    })
})

#### Text DataFrame 생성

In [10]:
df_valid = pd.DataFrame(data_textsum['validation'])
df_test = pd.DataFrame(data_textsum['test'])

In [17]:
df_valid['original_text'] = [row['post'] for row in df_valid['']]
df_test['original_text'] = [row['article'] for row in df_test['info']]

df_valid['sum_text'] = [row['text'] for row in df_valid['summary']]
df_test['sum_text'] = [row['text'] for row in df_test['summary']]

df_textsum = pd.concat([df_valid[['original_text', 'sum_text']], df_test[['original_text', 'sum_text']]], ignore_index=True)

In [21]:
df_textsum.shape

(14897, 2)

In [22]:
df_textsum.head(2)

,original_text,sum_text
0,"Recently, my fiance (20 m) and I (19f) moved ...",Fiance and I recently got infected with scabi...
1,"Recently, my fiance (20 m) and I (19f) moved ...",my fiance and I refuse to treat our room mate...


### DataLoader

In [32]:
class Policy_Dataset(torch.utils.data.Dataset):

    def __init__(self, df_textsum): #, transforms_=None, random_masking = False,  unaligned=True ):
        
        self.original_text = df_textsum['original_text']
        self.sum_text = df_textsum['sum_text']

        print(f"My_dataset __init__ received : {self.original_text.shape}, {self.sum_text.shape}")
        print(f"Data Type : {type(self.original_text[0]), type(self.sum_text[0])}")
        print(f"Data example : {self.original_text[0], {self.sum_text[0]}}")

    def __getitem__(self, index):
        original_text = self.original_text[index]
        sum_text = self.sum_text[index]

        return sum_good_text, sum_bad_text


    def __len__(self):
        return len(self.sum_text)


In [30]:
# Batch size 2로 돌리면 Cuda out of memory 에러남 

batch_size = 2

In [31]:
train_loader = torch.utils.data.DataLoader(Policy_Dataset(df_textsum), batch_size=batch_size, shuffle=False, drop_last = False)
# valid_loader = torch.utils.data.DataLoader(Reward_Dataset(df_feedback), batch_size=batch_size, shuffle=False, drop_last = False)
# test_loader = torch.utils.data.DataLoader(Reward_Dataset(df_feedback), batch_size=batch_size, shuffle=False, drop_last = False)

My_dataset __init__ received : (14897,),
 (14897,)
Data Type : (<class 'str'>, <class 'str'>)
Data example : ("Recently, my fiance  (20 m) and I (19f) moved into a new apartment with a mutual friend (20m) and somehow contracted scabies (don't know how). We've both been itchy af and have been to the doctor who confirmed that it was scabies for the both of us. Our room mate (20m) has not had symptoms of scabies bites appear yet but I have asked him to get treated as well and to treat his clothes and linen so that our apartment does not get reinfested after treatment.\n\nMy room mate refuses to buy the lotion needed to kill the mites on his skin (if there are any on him) and refuses to rewash and dry his linen and clothes. I'm scared that if he does not get treated the infestation of our apartment will not go away. I'm almost there to asking him to move out if he refuses treatment . He is not on the lease.", {" Fiance and I recently got infected with scabies. Room mate refuses to get trea

## Policy Model

In [18]:
# class RewardModel(nn.Module):
#     def __init__(self, embed_size=1024, l1=512, l2=128, l3=32, l4=8):
#         super(RewardModel, self).__init__()
#         self.encoder = AutoModel.from_pretrained("allenai/led-large-16384-arxiv").encoder
#         self.pool = nn.AdaptiveMaxPool1d(1)
#         self.linear = nn.Sequential(
#             nn.Linear(embed_size, l1),
#             nn.ReLU(),
#             nn.Linear(l1, l2),
#             nn.ReLU(),
#             nn.Linear(l2, l3),
#             nn.ReLU(),
#             nn.Linear(l3, l4),
#             nn.ReLU(),
#             nn.Linear(l4, 1)
#         )

#     def forward(self, input_ids):
#         global_attention_mask = torch.zeros_like(input_ids)
#         global_attention_mask[:, 0] = 1
#         output = self.encoder(input_ids, global_attention_mask=global_attention_mask).last_hidden_state
#         output = output.permute(0, 2, 1)
#         output = self.pool(output).squeeze(-1)
        
#         output = self.linear(output)
#         return output.squeeze(1)

In [33]:
policy_model= PolicyModel()

NameError: name 'PolicyModel' is not defined

In [20]:
policy_model.to(device)

RewardModel(
  (encoder): LEDEncoder(
    (embed_tokens): Embedding(50265, 1024, padding_idx=1)
    (embed_positions): LEDLearnedPositionalEmbedding(16384, 1024)
    (layers): ModuleList(
      (0): LEDEncoderLayer(
        (self_attn): LEDEncoderAttention(
          (longformer_self_attn): LEDEncoderSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (query_global): Linear(in_features=1024, out_features=1024, bias=True)
            (key_global): Linear(in_features=1024, out_features=1024, bias=True)
            (value_global): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (output): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (activation_fn): G

## Training

In [22]:
optimizer = optim.AdamW(reward_model.parameters(), lr=0.001)
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss()
# criterion = nn.LogSigmoid()

In [23]:
def train(_optimizer = optimizer, _criterion = criterion, epoch = 1, policy_model = policy_model):
  
    loss_list = []
    reward_list = []

    policy_model.train() 

    for i in range(epoch):

        for index, (original_text, sum_text) in enumerate(train_loader):
            
            original_token = tokenizer.batch_encode_plus(original_text, padding=True, return_tensors='pt').input_ids
            sum_token = tokenizer.batch_encode_plus(sum_text, padding=True, return_tensors='pt').input_ids
            
            original_token = original_token.to(device)
            sum_token = sum_token.to(device)
            
            sum_output = policy_model(original_token)

            # MSE Loss
            # loss = criterion(good_reward, bad_reward)

            # CrossEntropy
            loss =criterion(sum_output, original_token)

            # BCELoss 
            # loss = criterion(nn.softmax(pred_r), target_r)
            # loss = criterion.(pred_r, target_r)

            # LogSigmoid 
            # loss = criterion(traget_r - pred_r)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()
            
            
            print(f"Batch {index} \t Loss : {loss:} ")

    # return loss_list, reward_list



In [24]:
train()

Batch 0 	 Loss : 1.3625036388020817e-07 
Batch 1 	 Loss : 1.4877531384627218e-06 
Batch 2 	 Loss : 2.777888710170373e-07 
Batch 3 	 Loss : 1.9376248161506737e-08 
Batch 4 	 Loss : 2.271854704360976e-08 
Batch 5 	 Loss : 2.3390875014683843e-08 
Batch 6 	 Loss : 3.895446809565328e-08 
Batch 7 	 Loss : 1.4166980477625657e-08 
Batch 8 	 Loss : 7.431962245618706e-08 
Batch 9 	 Loss : 2.2598777960070038e-08 
Batch 10 	 Loss : 4.8292971399632734e-08 
Batch 11 	 Loss : 1.7410187069799576e-08 
Batch 12 	 Loss : 1.232551127827719e-08 
Batch 13 	 Loss : 4.763728611578699e-08 
Batch 14 	 Loss : 3.747131671616444e-08 
Batch 15 	 Loss : 7.477436270164617e-08 
Batch 16 	 Loss : 8.197734047143967e-09 
Batch 17 	 Loss : 4.1115356452792184e-08 
Batch 18 	 Loss : 3.344041488162475e-08 
Batch 19 	 Loss : 4.748117454766998e-10 
Batch 20 	 Loss : 1.09113829083185e-09 
Batch 21 	 Loss : 4.9076724906171876e-08 
Batch 22 	 Loss : 3.9041658794758405e-10 
Batch 23 	 Loss : 7.378486266418349e-09 
Batch 24 	 Loss 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Batch 10293 	 Loss : 1.1102230246251565e-15 
Batch 10294 	 Loss : 1.1102230246251565e-16 
Batch 10295 	 Loss : 2.220446049250313e-16 
Batch 10296 	 Loss : 2.220446049250313e-16 
Batch 10297 	 Loss : 4.440892098500626e-16 
Batch 10298 	 Loss : 2.220446049250313e-16 
Batch 10299 	 Loss : 2.220446049250313e-15 
Batch 10300 	 Loss : 5.551115123125783e-16 
Batch 10301 	 Loss : 1.887379141862766e-15 
Batch 10302 	 Loss : 5.551115123125783e-16 
Batch 10303 	 Loss : 1.609823385706477e-14 
Batch 10304 	 Loss : 1.1102230246251565e-16 
Batch 10305 	 Loss : 2.220446049250313e-16 
Batch 10306 	 Loss : 6.439293542825908e-15 
Batch 10307 	 Loss : 1.4432899320127035e-15 
Batch 10308 	 Loss : 1.4432899320127035e-15 
Batch 10309 	 Loss : 1.887379141862766e-15 
Batch 10310 	 Loss : 1.1102230246251565e-15 
Batch 10311 	 Loss : 8.881784197001252e-16 
Batch 10312 	 Loss : 1.887379141862766e-15 
Batch 10313 	 Loss : 4.440892098500626e-16 
Batch 10314 	 Loss : 1.1102230246251565e-14 
Batch 10315 	 Loss : 5.55

KeyboardInterrupt: 

----------------------

----


## GPU 초기화 

In [40]:
# !pip install numba

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 8.4 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 6.8 MB/s eta 0:00:0000:0100:01


In [25]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [26]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda
